# 0. 기본 정보
* 코드 작성자: 여서연
* 코드 작성일: 2024-05-23

# 1. 기초 설정

## 사용 라이브러리

In [1]:
import requests
import json
import xmltodict
import pandas as pd

from dotenv import load_dotenv
import os

## 기타 설정

In [2]:
# 데이터 프레임 열 전부 표시
pd.set_option('display.max_columns', None)

In [5]:
load_dotenv()
myKEY = os.environ.get('PUBLIC_DATA_API_KEY')

# 2. API 데이터 수집

## A. [환경부 국립환경과학원_골프장 농약사용량 DB](https://www.data.go.kr/data/15000937/openapi.do)

In [6]:
# 2020년 전국 데이터
golf_url = 'http://apis.data.go.kr/1480523/SgisGp/getSgisGpList1'
golf_params = {'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1000', 'resultType' : 'JSON', 'search_year': '2020'}

golf_response = requests.get(golf_url, params=golf_params)
#golf_response.content

In [7]:
golf_result = json.loads(golf_response.content)
#golf_result['getSgisGpList']['header']

In [8]:
golf = pd.DataFrame(golf_result['getSgisGpList']['item'])
golf

,SPOT_STD_CODE,YEAR,GOLFCLUB_NAME,ADDRESS,HOLE_CNT,SIDO,REG_YEAR,GOLF_TYPE_NAME,TOTAL_GROUND,PES_GROUND,NONE_GROUND,SPRAY_SUPPLY,REAL_SUPPLY,HA_REAL,HA_SPRAY,THA_REAL,THA_SPRAY
0,42760G040100003,2020,용평리조트GC,강원도 평창군 대관령면 용산리 130,18,강원도,1989,회원제,1007011,742109,264902,1021.93,415.69389,5.601520666101611,13.770618601849593,4.127997509461168,10.148151311157475
1,42760G040100002,2020,용평버치힐GC,강원도 평창군 대관령면 용산리 130,18,강원도,2004,회원제,983642,613936,369706,1427.67,311.60783,5.0755751413828145,23.25437830653358,3.167898788380325,14.514122007803653
2,42730G040100001,2020,웰리힐리컨트리클럽,강원도 횡성군 둔내면 우용리 15,36,강원도,2007,회원제,1899828,1175910,723918,2206.4399999999996,726.8172500000001,6.180891819952208,18.763680893945963,3.825700273919534,11.613893468250808
3,42130G040200004,2020,인터불고호텔원주G.C.,강원도 원주시 반곡동 1809-1,9,강원도,2010,대중제,165318,116705,48613,368.423,83.396624,7.145934107364723,31.56874169915599,5.044618492844094,22.285716013985166
4,42110G040100001,2020,제이드팰리스골프클럽,강원도 춘천시 남산면 서천리 121-19,18,강원도,2004,회원제,986626,509951,476675,1293.25,233.1966,4.572921712086063,25.36027971314891,2.3635764717329564,13.107803767587718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,42130G040100002,2020,오크힐스CC,강원도 원주시 지정면 판대리 577-1,18,강원도,2007,회원제,929669,476512,453157,217.91000000000003,57.95505000000001,1.2162348482304748,4.573022295346182,0.6233944554459706,2.343952525038482
537,42190G040100001,2020,오투리조트CC,강원도 태백시 황지동 산 176-28,18,강원도,2008,대중제,1045231,935739,109492,278.9,110.5605,1.1815313885602716,2.980531964575592,1.0577613943711965,2.66830968465344
538,42190G040200001,2020,오투리조트골프,강원도 태백시 황지동 산 176-28,9,강원도,2008,대중제,457558,405814,51744,141.6,55.89439,1.3773401114796433,3.489283267703923,1.2215804335188107,3.094689634975238
539,42730G040100003,2020,옥스필드 CC,강원도 횡성군 서원면 유현리 638,18,강원도,2010,대중제,1048906,571260,477646,270.468,77.43168,1.355454259006407,4.734586703077408,0.7382137198185538,2.5785723410868084


In [9]:
golf.to_csv('../data/golf.csv', index=False, encoding='utf-8-sig')

In [10]:
unique_addresses = golf.groupby('SIDO')['ADDRESS'].nunique().reset_index()
unique_addresses

,SIDO,ADDRESS
0,강원도,53
1,경기도,145
2,경상남도,40
3,경상북도,50
4,광주광역시,4
5,대구광역시,4
6,대전광역시,4
7,부산광역시,12
8,서울특별시,2
9,세종특별자치시도,3


## B. [기상청_지상(종관, ASOS)기상관측자료](https://www.data.go.kr/data/15043648/fileData.do)

In [11]:
rain_summer = pd.read_csv('../data/OBS_ASOS_MNH_20240524000233.csv', encoding='cp949')
rain_winter = pd.read_csv('../data/OBS_ASOS_MNH_20240524000245.csv', encoding='cp949')

In [12]:
rain_summer.head()

,지점,지점명,일시,월합강수량(00~24h만)(mm),일최다강수량(mm),1시간최다강수량(mm),10분최다강수량(mm),일최다강수량 나타난날(yyyymmdd),1시간최다강수량 나타난날(yyyymmdd),10분최다강수량 나타난날(yyyymmdd),최심적설(cm),최심신적설(cm),3시간신적설합(cm),최심적설 나타난날(yyyymmdd),최심신적설 나타난날(yyyymmdd)
0,90,속초,2020-07,374.7,115.5,35.9,8.0,20200724,20200724,20200724,NaN,NaN,NaN,NaN,NaN
1,90,속초,2020-08,411.3,127.9,31.3,8.4,20200805,20200804,20200804,NaN,NaN,NaN,NaN,NaN
2,93,북춘천,2020-07,183.6,60.9,17.0,5.6,20200723,20200714,20200722,NaN,NaN,NaN,NaN,NaN
3,93,북춘천,2020-08,821.2,188.5,72.5,18.8,20200803,20200803,20200803,NaN,NaN,NaN,NaN,NaN
4,95,철원,2020-07,184.5,48.5,19.9,14.6,20200723,20200718,20200718,NaN,NaN,NaN,NaN,NaN


In [13]:
rain_winter.head()

,지점,지점명,일시,월합강수량(00~24h만)(mm),일최다강수량(mm),1시간최다강수량(mm),10분최다강수량(mm),일최다강수량 나타난날(yyyymmdd),1시간최다강수량 나타난날(yyyymmdd),10분최다강수량 나타난날(yyyymmdd),최심적설(cm),최심신적설(cm),3시간신적설합(cm),최심적설 나타난날(yyyymmdd),최심신적설 나타난날(yyyymmdd)
0,90,속초,2020-01,148.1,58.6,NaN,NaN,20200107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90,속초,2020-02,55.8,24.1,NaN,NaN,20200225,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93,북춘천,2020-01,55.9,37.5,NaN,NaN,20200107,NaN,NaN,0.4,0.4,1.0,20200106.0,20200106.0
3,93,북춘천,2020-02,35.3,19.4,NaN,NaN,20200225,NaN,NaN,1.8,1.8,2.0,20200205.0,20200204.0
4,95,철원,2020-01,79.7,52.3,NaN,NaN,20200107,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## C. [환경부 국립환경과학원_수질 DB](https://www.data.go.kr/data/15081073/openapi.do)

### 시도별 수질측정망 지점

In [14]:
code = pd.read_excel('../API documents/국립환경과학원_openAPI_활용가이드_수질DB/물환경 수질측정망 운영결과 DB_물환경_코드_코드명.xlsx', header=1)
code

,구분,지점코드,지점명,조사기관,주소,대권역,중권역
0,하천수,1001A05,송천1,원주지방환경청,강원도 평창군 대관령면 용산리(용산교),한강,남한강상류
1,하천수,1001A10,송천4,원주지방환경청,강원도 정선군 여량면 유천리(송천교),한강,남한강상류
2,하천수,1001A15,골지천2,한국수자원공사,강원도 태백시 하사미동 290-1(하사미교),한강,남한강상류
3,하천수,1001A20,번천,한국수자원공사,강원도 삼척시 하장면 숙암리 90-4,한강,남한강상류
4,하천수,1001A25,골지천4,원주지방환경청,강원도 정선군 여량면 여량리(여량1교),한강,남한강상류
...,...,...,...,...,...,...,...
2213,하천수,5101A05,탐진강1(탐진A1),영산강물환경연구소,전라남도 장흥군 유치면 운월리(운월교),영산강ㆍ섬진강,탐진강
2214,하천수,5101A15,유치천1(탐진A2),영산강물환경연구소,전라남도 영암군 금정면 연소리(한대교),영산강ㆍ섬진강,탐진강
2215,하천수,5101A25,옴천천2(탐진A3),영산강물환경연구소,전라남도 강진군 옴천면 봉림리(봉림교),영산강ㆍ섬진강,탐진강
2216,하천수,5101A45,금강천1(탐진B1),영산강물환경연구소,전라남도 강진군 병영면 삭양리(삭양교),영산강ㆍ섬진강,탐진강


In [15]:
# 주소 컬럼의 데이터를 공백을 기준으로 분리하여 시도 추출
code['시도'] = code['주소'].apply(lambda x: x.split()[0])
code['시도'].unique()

array(['강원도', '충청북도', '경기도', '서울특별시', '인천광역시', '충청남도', '인천광역시시', '경상북도',
       '경북', '대구광역시', '대구광역시시', '경상남도', '전라북도', '부산광역시', '울산광역시',
       '울산광역시시', '부산광역시시', '대전광역시', '대전광역시시', '세종특별자치시', '전라남도', '광주',
       '광주광역시', '제주특별자치도'], dtype=object)

In [16]:
codes = code[code['시도']=='경기도']['지점코드'].unique()
codes_list = list(codes)
comma_string = ','.join(codes_list)
comma_string

'1007A05,1007A10,1007A15,1007A20,1007A22,1007A25,1007A27,1007A40,1007A42,1007A44,1007A46,1007A50,1007A55,1007A59,1007A60,1007A65,1007A70,1007A75,1007B10,1007B20,1007D35,1007D40,1007D45,1007D50,1007D55,1007D60,1007D65,1007D70,1007D75,1007D80,1007D85,1007D90,1013A70,1013A80,1013A90,1013A95,1015A20,1015A25,1015A30,1015A40,1015A42,1015A47,1015A50,1015A60,1015B10,1015B20,1015B30,1015B40,1016A10,1016A20,1016A30,1016A40,1016A50,1016A60,1016A70,1016A80,1016B10,1016D10,1016D20,1017A10,1017B10,1018A06,1018A12,1018A14,1018A16,1018A28,1018A60,1018A66,1018A70,1018A72,1018D10,1019A10,1019A15,1019A20,1019A25,1019A30,1019A45,1019A50,1019A55,1019A60,1021A05,1021A10,1021B10,1022A10,1022A20,1022A25,1022A30,1022A35,1022A37,1022A40,1022A45,1022A50,1022A55,1022A65,1022A70,1022B15,1022D35,1022D40,1022D45,1022D50,1022D55,1022D60,1022D65,1022D70,1022D75,1022D80,1022E10,1022E20,1022E30,1022E40,1023A10,1023A20,1023A25,1023A30,1023A40,1023A50,1023D20,1023D30,1023D40,1023D50,1023D60,1023D70,1023E10,1024A10,1025A03

### 물환경 수질측정망 운영결과 월평균 DB

In [17]:
water_qual_url = 'http://apis.data.go.kr/1480523/WaterQualityService/getWaterMeasuringListMavg'
water_qual_params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '5000', 'resultType' : 'JSON', 'ptNoList' : comma_string, 'wmyrList' : '2020', 'wmodList' : '01,02,07,08,09' }

water_qual_response = requests.get(water_qual_url, params=water_qual_params)
#print(water_qual_response.content)

In [18]:
water_qual_result = json.loads(water_qual_response.content)
#water_qual_result['getWaterMeasuringListMavg']['item']

In [19]:
water_qual = pd.DataFrame(water_qual_result['getWaterMeasuringListMavg']['item'])
water_qual

,ROWNO,PTNM,PTNO,WMYR,WMOD,ITEMAMNT,ITEMTEMP,ITEMPH,ITEMDOC,ITEMBOD,ITEMCOD,ITEMSS,ITEMTCOLI,ITEMTN,ITEMTP,ITEMCD,ITEMCN,ITEMPB,ITEMCR6,ITEMAS,ITEMHG,ITEMCU,ITEMABS,ITEMPCB,ITEMOP,ITEMMN,ITEMTRANS,ITEMCLOA,ITEMZN,ITEMCR,ITEMFE,ITEMPHENOL,ITEMNHEX,ITEMEC,ITEMTCE,ITEMPCE,ITEMNO3N,ITEMNH3N,ITEMECOLI,ITEMPOP,ITEMDTN,ITEMDTP,ITEMFL,ITEMCOL,ITEMCCL4,ITEMDCM,ITEMBENZENE,ITEMCHCL3,ITEMTOC,ITEMDEHP,ITEMANTIMON,ITEMDIOX,ITEMHCHO,ITEMHCB,ITEMNI,ITEMBA,ITEMSE
0,1,곡수천1,1007A58,2020,01,0.151,6.9,8.4,11.8,1.1,2.5,1.5,NaN,1.981,0.024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,1.6,None,NaN,NaN,NaN,NaN,NaN,NaN,None
1,2,가평천2,1013A80,2020,01,NaN,3.9,6.5,13.2,1.7,1.3,0.8,1300.0,3.127,0.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,NaN,NaN,NaN,0.0,NaN,162,NaN,NaN,2.346,0.092,130.0,0.029,3.102,0.030,NaN,None,NaN,NaN,NaN,NaN,0.9,None,NaN,NaN,NaN,NaN,NaN,NaN,None
2,3,용담천,1007A63,2020,01,0.312,5.6,8.0,12.5,0.9,1.9,0.5,NaN,2.734,0.012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,0.9,None,NaN,NaN,NaN,NaN,NaN,NaN,None
3,4,조종천2,1015A25,2020,01,NaN,3.0,6.7,14.1,1.7,1.2,0.8,330.0,4.479,0.020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5,NaN,NaN,NaN,0.0,NaN,212,NaN,NaN,3.804,0.093,170.0,0.018,4.365,0.019,NaN,None,NaN,NaN,NaN,NaN,1.0,None,NaN,NaN,NaN,NaN,NaN,NaN,None
4,5,청평댐1,1015B30,2020,01,NaN,4.4,8.7,12.9,0.3,2.9,3.0,2000.0,2.356,0.013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.4,NaN,NaN,NaN,0.0,NaN,148,NaN,NaN,1.795,0.137,61.0,0.001,2.265,0.006,NaN,None,NaN,NaN,NaN,NaN,1.5,None,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,1129,구운천2,1015A42,2020,09,8.238,19.1,7.9,11.3,0.7,2.7,12.4,3100.0,2.376,0.055,0.0,0.0,0.0,0.0,0.0,0.000005,NaN,0.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,0.0,NaN,126,NaN,NaN,2.147,0.023,1300.0,0.026,2.340,0.035,NaN,None,NaN,NaN,NaN,NaN,1.8,None,0.0,NaN,NaN,NaN,NaN,NaN,None
1129,1130,도심천,1025A18,2020,09,NaN,22.7,7.9,8.3,1.7,4.6,1.3,46000.0,5.000,0.288,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,0.0,NaN,NaN,NaN,NaN,2.4,NaN,NaN,NaN,0.0,NaN,279,NaN,NaN,4.200,0.240,28000.0,0.200,4.700,0.220,NaN,None,NaN,NaN,NaN,NaN,2.8,None,0.0,NaN,NaN,NaN,NaN,NaN,None
1130,1131,궁촌천,1025A15,2020,09,NaN,20.5,7.5,8.7,0.6,3.6,11.3,42000.0,3.100,0.054,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,0.0,NaN,NaN,NaN,NaN,3.1,NaN,NaN,NaN,0.0,NaN,106,NaN,NaN,2.500,0.000,3900.0,0.100,3.100,0.042,NaN,None,NaN,NaN,NaN,NaN,1.7,None,0.0,NaN,NaN,NaN,NaN,NaN,None
1131,1132,도곡,1025A12,2020,09,NaN,21.0,7.8,9.0,0.6,3.0,10.6,21000.0,2.125,0.056,0.0,0.0,0.0,0.0,0.0,0.000000,NaN,0.0,NaN,NaN,NaN,NaN,8.8,NaN,NaN,NaN,0.0,NaN,133,NaN,NaN,1.867,0.003,170.0,0.022,2.102,0.033,NaN,None,NaN,NaN,NaN,NaN,1.7,None,0.0,NaN,NaN,NaN,NaN,NaN,None


In [20]:
water_qual.to_csv('../data/water_quality_Gyeonggi.csv', index=False, encoding='utf-8-sig')

# X. 하단 코드는 사용하지 않습니다.

### 하수도 일일 수질 정보 조회 서비스

In [21]:
sewage_url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/sewerage/waterQuality/fcltylist/codelist'
sewage_params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '100' }

sewage_response = requests.get(sewage_url, params=sewage_params)
print(sewage_response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><loccd>48000</loccd><locnm>\xeb\x8a\xa5\xea\xb3\xa1\xea\xb3\xb5\xea\xb3\xb5\xed\x95\x98\xec\x88\x98\xec\xb2\x98\xeb\xa6\xac\xec\x8b\x9c\xec\x84\xa4</locnm></item><item><loccd>51730</loccd><locnm>\xeb\x91\x94\xeb\x82\xb4\xea\xb3\xb5\xea\xb3\xb5\xed\x95\x98\xec\x88\x98\xec\xb2\x98\xeb\xa6\xac\xec\x8b\x9c\xec\x84\xa4(\xec\x82\xac\xec\x97\x85\xec\xa2\x85\xeb\xa3\x8c)</locnm></item><item><loccd>48000</loccd><locnm>\xec\x8b\x9c\xed\x99\x94\xea\xb3\xb5\xea\xb3\xb5\xed\x95\x98\xec\x88\x98\xec\xb2\x98\xeb\xa6\xac\xec\x8b\x9c\xec\x84\xa4</locnm></item><item><loccd>4710</loccd><locnm>\xec\x95\x84\xec\x82\xb0\xec\x8b\xa0\xeb\x8f\x84\xec\x8b\x9c\xea\xb3\xb5\xea\xb3\xb5\xed\x95\x98\xec\x88\x98\xec\xb2\x98\xeb\xa6\xac\xec\x8b\x9c\xec\x84\xa4</locnm></item><item><loccd>47750</loccd><locnm>\xec\x95\x88\xeb\x8d\x95\xea\xb3\xb5\xea\xb3\xb5

In [22]:
sewage_dict = xmltodict.parse(sewage_response.content)
sewage_json = json.dumps(sewage_dict, ensure_ascii=False)
sewage_result = json.loads(sewage_json)
sewage_result['response']['body']['items']['item']

[{'loccd': '48000', 'locnm': '능곡공공하수처리시설'},
 {'loccd': '51730', 'locnm': '둔내공공하수처리시설(사업종료)'},
 {'loccd': '48000', 'locnm': '시화공공하수처리시설'},
 {'loccd': '4710', 'locnm': '아산신도시공공하수처리시설'},
 {'loccd': '47750', 'locnm': '안덕공공하수처리시설'},
 {'loccd': '47850', 'locnm': '약목하수처리시설'},
 {'loccd': '47000', 'locnm': '연서공공하수처리시설'},
 {'loccd': '47850', 'locnm': '왜관(기존)하수처리시설'},
 {'loccd': '47850', 'locnm': '왜관(증설)하수처리시설'},
 {'loccd': '51730', 'locnm': '우항공공하수처리시설(사업종료)'},
 {'loccd': '47111', 'locnm': '장량공공하수처리시설'},
 {'loccd': '44770', 'locnm': '장항공공하수처리시설'},
 {'loccd': '47750', 'locnm': '진보공공하수처리시설'},
 {'loccd': '47750', 'locnm': '청송공공하수처리시설'},
 {'loccd': '51730', 'locnm': '횡성공공하수처리시설(사업종료)'}]

In [23]:
sewage = pd.DataFrame(sewage_result['response']['body']['items']['item'])
sewage

,loccd,locnm
0,48000,능곡공공하수처리시설
1,51730,둔내공공하수처리시설(사업종료)
2,48000,시화공공하수처리시설
3,4710,아산신도시공공하수처리시설
4,47750,안덕공공하수처리시설
5,47850,약목하수처리시설
6,47000,연서공공하수처리시설
7,47850,왜관(기존)하수처리시설
8,47850,왜관(증설)하수처리시설
9,51730,우항공공하수처리시설(사업종료)


### 하수도 일일 수질 정보 조회 서비스

In [24]:
water_qual_url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/sewerage/waterQuality/day/list'

In [25]:
# 2020년 7월 ~ 8월
summer_params = {'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '100', 'stDt' : '2020-07-01', 'edDt' : '2020-08-31'}

summer_response = requests.get(water_qual_url, params=summer_params)
print(summer_response.content)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><bBac>220</bBac><bBod>1.6</bBod><bCod>9.1</bCod><bSs>3.6</bSs><bTn>1.342</bTn><bTp>0.522</bTp><bVal>4,712</bVal><loccd>48000</loccd><locnm>\xeb\x8a\xa5\xea\xb3\xa1\xea\xb3\xb5\xea\xb3\xb5\xed\x95\x98\xec\x88\x98\xec\xb2\x98\xeb\xa6\xac\xec\x8b\x9c\xec\x84\xa4</locnm><temper>17.4</temper><uBac>448,000</uBac><uBod>486.8</uBod><uCod>80.4</uCod><uSs>320.0</uSs><uTn>59.533</uTn><uTp>5.611</uTp><uVal>5,145</uVal><weather>\xeb\xb9\x84</weather><wqdt>2020-07-01</wqdt><wtemper>24</wtemper></item><item><bBac>110</bBac><bBod>1.1</bBod><bCod>7.7</bCod><bSs>2.4</bSs><bTn>1.536</bTn><bTp>0.786</bTp><bVal>4,447</bVal><loccd>48000</loccd><locnm>\xeb\x8a\xa5\xea\xb3\xa1\xea\xb3\xb5\xea\xb3\xb5\xed\x95\x98\xec\x88\x98\xec\xb2\x98\xeb\xa6\xac\xec\x8b\x9c\xec\x84\xa4</locnm><temper>16.9</temper><uBac>392,000</uBac><uBod>276.0</uBod><uCod>86

In [26]:
summer_dict = xmltodict.parse(summer_response.content)
summer_json = json.dumps(summer_dict, ensure_ascii=False)
summer_result = json.loads(summer_json)
summer_result['response']['body']['items']['item']

[{'bBac': '220',
  'bBod': '1.6',
  'bCod': '9.1',
  'bSs': '3.6',
  'bTn': '1.342',
  'bTp': '0.522',
  'bVal': '4,712',
  'loccd': '48000',
  'locnm': '능곡공공하수처리시설',
  'temper': '17.4',
  'uBac': '448,000',
  'uBod': '486.8',
  'uCod': '80.4',
  'uSs': '320.0',
  'uTn': '59.533',
  'uTp': '5.611',
  'uVal': '5,145',
  'weather': '비',
  'wqdt': '2020-07-01',
  'wtemper': '24'},
 {'bBac': '110',
  'bBod': '1.1',
  'bCod': '7.7',
  'bSs': '2.4',
  'bTn': '1.536',
  'bTp': '0.786',
  'bVal': '4,447',
  'loccd': '48000',
  'locnm': '능곡공공하수처리시설',
  'temper': '16.9',
  'uBac': '392,000',
  'uBod': '276.0',
  'uCod': '86.9',
  'uSs': '257.5',
  'uTn': '61.949',
  'uTp': '5.782',
  'uVal': '4,909',
  'weather': '흐림',
  'wqdt': '2020-07-02',
  'wtemper': '25'},
 {'bBac': '220',
  'bBod': '1.4',
  'bCod': '9.7',
  'bSs': '1.5',
  'bTn': '1.413',
  'bTp': '0.508',
  'bVal': '4,608',
  'loccd': '48000',
  'locnm': '능곡공공하수처리시설',
  'temper': '20.1',
  'uBac': '504,000',
  'uBod': '276.0',
  'uCod': 

In [27]:
water_qual_summer = pd.DataFrame(summer_result['response']['body']['items']['item'])
water_qual_summer

,bBac,bBod,bCod,bSs,bTn,bTp,bVal,loccd,locnm,temper,uBac,uBod,uCod,uSs,uTn,uTp,uVal,weather,wqdt,wtemper
0,220,1.6,9.1,3.6,1.342,0.522,"4,712",48000,능곡공공하수처리시설,17.4,"448,000",486.8,80.4,320.0,59.533,5.611,"5,145",비,2020-07-01,24
1,110,1.1,7.7,2.4,1.536,0.786,"4,447",48000,능곡공공하수처리시설,16.9,"392,000",276.0,86.9,257.5,61.949,5.782,"4,909",흐림,2020-07-02,25
2,220,1.4,9.7,1.5,1.413,0.508,"4,608",48000,능곡공공하수처리시설,20.1,"504,000",276.0,88.7,195.0,55.024,5.953,"5,156",비,2020-07-03,25
3,110,2.6,9.5,4.1,1.691,0.689,"4,325",48000,능곡공공하수처리시설,19,"336,000",251.4,98.7,180.0,67.827,6.273,"4,769",비,2020-07-04,25
4,170,3.9,9.0,3.1,1.949,0.713,"4,426",48000,능곡공공하수처리시설,20.2,"504,000",348.6,96.0,157.5,71.117,6.655,"4,935",흐림,2020-07-05,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6,1.2,3.6,1.0,7.452,0.043,955,51730,둔내공공하수처리시설(사업종료),24.6,"82,000",75.2,85.0,106.0,25.230,3.252,"1,195",비,2020-08-04,23
96,8,1.1,3.8,1.2,7.596,0.038,937,51730,둔내공공하수처리시설(사업종료),23.8,"82,000",81.2,82.5,96.0,27.150,3.396,"1,136",비,2020-08-05,23
97,6,1.2,4.0,3.2,7.164,0.204,895,51730,둔내공공하수처리시설(사업종료),23.9,"89,000",85.6,87.5,114.0,26.610,1.656,"1,150",비,2020-08-06,22
98,5,1.3,3.2,3.6,6.324,0.202,750,51730,둔내공공하수처리시설(사업종료),23.8,"78,000",81.6,82.5,120.0,25.890,1.836,975,비,2020-08-07,21


In [28]:
# 2020년 1월 ~ 2월
winter_params ={'serviceKey' : key, 'pageNo' : '1', 'numOfRows' : '1000', 'stDt' : '2020-01-01', 'edDt' : '2020-02-29'}